In [126]:
#-*- coding: utf-8 -*-

import os

mingw_path = r'C:\Program Files\mingw-w64\x86_64-7.1.0-posix-seh-rt_v5-rev0\mingw64\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

from email import policy
from email.parser import BytesParser
from email.parser import Parser

import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english', ignore_stopwords=True)
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-z]{3,}')

from string import punctuation
from collections import defaultdict
from heapq import nlargest

from __future__ import print_function
import re
import pandas as pd
import numpy as np
from os import listdir
from collections import Counter
from bs4 import BeautifulSoup

import sklearn
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

import xgboost as xgb
import math 
import gensim

from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib import pylab
%matplotlib inline

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:54.180000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [127]:
docs_path = r'C:\Users\Artur_Zahreba\Desktop\WorkFusion\P&G\sample email June 15\Docs'

doc_types = sorted(listdir(docs_path))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:54.993000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [128]:
def getDoc(filePath):
    with open(filePath, 'rb') as fp:
        msg = BytesParser(policy=policy.default).parse(fp)
        msg_text = msg.get_body(preferencelist=('html', 'plain')).get_content()         
        
        html_doc = msg_text
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        texts = []
        
        for block in soup.findAll('p'):
            texts.append(block.text)
            
        text = '\n'.join(texts).encode('ascii', 'replace').decode('utf-8').replace("?","").strip()

        fp.close()
        
        return (text, msg)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:55.533000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [129]:
doc_texts = []
doc_msgs = []

for doc_type in doc_types:
    
    filenames = sorted(listdir(docs_path + '\\' + doc_type))
    filenames = filter(lambda x: re.search(r'\.msg$', x), filenames)
    
    for filename in filenames:
        filePath = docs_path + '\\' + doc_type + '\\' + filename
        text, msg = getDoc(filePath)
        doc_texts.append((doc_type, filename, text))
        doc_msgs.append(msg)
            
print(len(doc_texts))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:57.383000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


206


In [130]:
doc_texts[0]

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:58.515000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


('Approved',
 'FW   Approval  BC OND JFM allowance.msg',
 'Hi OM \n\nPlease help to process. \n\n\n\nRegards\nEunice\n\n\nThe Metropolis, 11 North Buona Vista Drive, #21-07, The Metropolis Tower 2, Singapore 138589\n\nAll decisions on pricing, promotion, distribution, assortment and shelving are at the sole discretion of the retailer.\nThis electronic message transmission contains information which may be confidential. The information is intended for the use of the individual or entity named above. If you are not the intended recipient, and have received this electronic transmission in error, please notify sender then delete immediately.\n\n\nFrom: Meng, FanRu Sent: Wednesday, April 26, 2017 7:07 PMTo: Eunice, Chiam <eunice.ec@pg.com>Subject: RE: [Approval] BC OND JFM allowance\n\n\nApproved. \n\n\nBest Regards,\nFanRu MENG\n\nFrom: Eunice, Chiam Sent: Wednesday, April 26, 2017 6:01 PMTo: Meng, FanRu <meng.fa.1@pg.com>Subject: FW: [Approval] BC OND JFM allowance\n\nHI Fanru \n\nCan you

In [131]:
lables, _, texts = zip(*doc_texts)
joblib.dump((lables, texts), 'labels_texts.pkl')

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:32:06.500000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


['labels_texts.pkl']

In [106]:
lables = list(lables)
texts = list(texts)
# texts = list(map(lambda x: stemmer.stem(x), list(texts)))
# texts = list(map(lambda x: tokenizer.tokenize(x), list(texts)))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.178000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [107]:
lable_dict = dict(zip(list(set(lables)), range(len(set(lables)))))
inv_lable_dict = dict(zip(lable_dict.values(), lable_dict.keys()))
print('Number of classes:', len(inv_lable_dict))
print(inv_lable_dict)

Number of classes: 2
{0: 'Other', 1: 'Approved'}


C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.183000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [108]:
lables_int = list(map(lambda x: lable_dict[x], lables))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.188000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [109]:
train_ind, test_ind = train_test_split(range(len(texts)), test_size = 0.2, random_state = 42)

X = np.array(texts)
y = np.array(lables_int)

X_train = X[train_ind]
X_test = X[test_ind]
y_train = y[train_ind]
y_test = y[test_ind]
#, X_test, y_train, y_test = train_test_split(texts, lables_int, test_size=0.20, random_state=0)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.192000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [110]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=False, max_df=0.8, min_df=5, use_idf=True, 
                                                             stop_words='english', lowercase=True, max_features=None,
                                                             strip_accents = 'unicode', ngram_range=(1,2), norm=u'l2',
                                                             smooth_idf=True, token_pattern=r'[a-z\d]{2,}')
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test =  vectorizer.transform(X_test)
X_train.shape

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.196000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


(164, 2200)

In [111]:
joblib.dump((X_train, X_test, y_train, y_test), 'xgb_data.pkl')

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.201000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


['xgb_data.pkl']

In [112]:
X_tr, X_ts, y_tr, y_ts = joblib.load('xgb_data.pkl')
X_tr

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.208000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


<164x2200 sparse matrix of type '<class 'numpy.float64'>'
	with 22820 stored elements in Compressed Sparse Row format>

In [113]:
D_train = xgb.DMatrix(X_train, label = y_train)
D_test = xgb.DMatrix(X_test, label = y_test)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.217000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [114]:
param = {}
param ['booster'] = 'gbtree'
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['gamma'] = 0.2
param['min_child_weight'] = 1.0
#param['scale_pos_weight'] = 1.0*(y_train.shape[0] - y_train.sum())/(y_train.sum()+1)
#param['max_delta_step'] = 10
param['max_depth'] = 2
param['silent'] = 0
param['subsample'] = 0.5
param['colsample_bytree' ] = 0.5
param['colsample_bylevel' ] = 0.5
param['alpha' ] = 0.0
param['lambda' ] = 5.0
param['nthread'] = 4
param['num_class'] = len(lable_dict)
# param['one_drop'] = 1
# param['rate_drop'] =  0.2
param['updater']='grow_histmaker,refresh'
# param['sample_type'] =  "weighted"
# param['normalize_type'] = "forest"
param['eval_metric'] = 'merror'
num_round = 200

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.221000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [115]:
watchlist  = [(D_test,'eval'), (D_train,'eval')]
bst = xgb.train(param, D_train, num_round, watchlist)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.226000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[0]	eval-merror:0.190476	eval-merror:0.164634
[1]	eval-merror:0.190476	eval-merror:0.164634
[2]	eval-merror:0.190476	eval-merror:0.164634
[3]	eval-merror:0.190476	eval-merror:0.164634
[4]	eval-merror:0.190476	eval-merror:0.164634
[5]	eval-merror:0.190476	eval-merror:0.164634
[6]	eval-merror:0.190476	eval-merror:0.164634
[7]	eval-merror:0.190476	eval-merror:0.164634
[8]	eval-merror:0.190476	eval-merror:0.164634
[9]	eval-merror:0.190476	eval-merror:0.164634
[10]	eval-merror:0.190476	eval-merror:0.164634
[11]	eval-merror:0.190476	eval-merror:0.164634
[12]	eval-merror:0.190476	eval-merror:0.164634
[13]	eval-merror:0.190476	eval-merror:0.164634
[14]	eval-merror:0.190476	eval-merror:0.164634
[15]	eval-merror:0.190476	eval-merror:0.164634
[16]	eval-merror:0.190476	eval-merror:0.164634
[17]	eval-merror:0.190476	eval-merror:0.164634
[18]	eval-merror:0.190476	eval-merror:0.164634
[19]	eval-merror:0.190476	eval-merror:0.164634
[20]	eval-merror:0.190476	eval-merror:0.164634
[21]	eval-merror:0.1904

In [116]:
xxx = bst.predict(D_test)

accuracy = 1.0 * list(map(lambda x, y: 1 if x == y else 0, y_test, xxx)).count(1) / len(y_test)
print(accuracy)

0.8095238095238095


C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.230000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [117]:
def softmax(array):
    e = map(lambda x: sum(list(map(lambda y: math.pow(math.e, y), x))), array)
    array = map(lambda x, y: list(map(lambda z: math.pow(math.e, z)/y, x)), array, e)
    return np.array(array)

def decision (y):
    m = max(y)
    cc = 0
    for i in y:
        if i == m:
            return cc
        cc +=1

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.238000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [118]:
bst_probs = softmax(bst.predict(D_test, output_margin=True))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.242000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


### CV

In [119]:
texts = list(map(lambda x: stemmer.stem(x), list(texts)))
texts = list(map(lambda x: tokenizer.tokenize(x), texts))
# texts = list(map(lambda x: re.sub(r'\s[ -\.\d]+\s', ' ', ' '.join(x)), texts))

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.247000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [120]:
# texts = map(lambda x: re.sub(r'(?<=[^\w])20\d\d(?=[^\w]?)', 'year_mask', x), texts)
# texts = map(lambda x: re.sub(r'year_mask +(0[1-9]|1[0-2]) +([0-2][1-9]|3[01])', 'date_mask', x), texts)
# texts = map(lambda x: re.sub(r'year_mask +([0-2][1-9]|3[01]) +(0[1-9]|1[0-2])', 'date_mask', x), texts)
# texts = map(lambda x: re.sub(r'([0-2][1-9]|3[01]) +(0[1-9]|1[0-2]) +year_mask', 'date_mask', x), texts)
# texts = map(lambda x: re.sub(r'(0[1-9]|1[0-2]) +([0-2][1-9]|3[01]) +year_mask', 'date_mask', x), texts)

lable_dict = dict(zip(list(set(lables)), range(len(set(lables)))))
inv_lable_dict = dict(zip(lable_dict.values(), lable_dict.keys()))

lables_int = np.array(list(map(lambda x: lable_dict[x], lables)))

# vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=False, max_df=1.0, min_df=5, use_idf=True, 
#                                                              stop_words='english', lowercase=True, max_features=None,
#                                                              strip_accents = 'unicode', ngram_range=(1,3), norm=u'l2',
#                                                              smooth_idf=True, token_pattern=r'[a-z]{3,}')

# X = vectorizer.fit_transform(texts)
                      
X = np.array(texts)
y = lables_int

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.250000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [121]:
print(X.shape)
print('# classes:', len(lable_dict))
for i in lable_dict.keys():
    print(i, ' '*(10 - len(i)), ':', list(lables_int).count(lable_dict[i]))

(206,)
# classes: 2
Other       : 35
Approved    : 171


C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.254000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [122]:
print(lable_dict)

{'Other': 0, 'Approved': 1}


C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.259000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [123]:
cv = sklearn.model_selection.KFold(n_splits=5, random_state=42, shuffle=True)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.262000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [124]:
param = {}
param ['booster'] = 'gbtree'
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['gamma'] = 0.1
param['min_child_weight'] = 0.1
#param['scale_pos_weight'] = 1.0*(y_train.shape[0] - y_train.sum())/(y_train.sum()+1)
#param['max_delta_step'] = 10
param['max_depth'] = 5
param['silent'] = 1
param['subsample'] = 0.5
param['colsample_bytree' ] = 0.5
param['colsample_bylevel' ] = 0.5
param['alpha' ] = 0.0
param['lambda' ] = 5.0
param['nthread'] = 4
param['num_class'] = len(lable_dict)
# param['one_drop'] = 1
# param['rate_drop'] =  0.2
param['updater']='grow_histmaker,prune'
# param['sample_type'] =  "weighted"
# param['normalize_type'] = "forest"
param['eval_metric'] = 'merror'
num_round = 200

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.267000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [125]:
cc = 0
accs = []
incorrect_classes = Counter()
incorrect_files = Counter()
common_mistakes = Counter()
for i, j in cv.split(range(len(doc_texts))):
    print ('Fold', cc, '...')
    X_train, X_test = X[i], X[j]
    y_train, y_test = y[i], y[j]
    
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=False, max_df=0.8, min_df=5, use_idf=True, 
                                                             stop_words='english', lowercase=True, max_features=None,
                                                             strip_accents = 'unicode', ngram_range=(1,3), norm=u'l2',
                                                             smooth_idf=True, token_pattern=r'[a-z\d]{2,}')
    vectorizer.fit(X_train)
    X_train = vectorizer.transform(X_train)
    X_test =  vectorizer.transform(X_test)
    
    D_train = xgb.DMatrix(X_train, label = y_train)
    D_test = xgb.DMatrix(X_test, label = y_test)
    
    bst = xgb.train(param, D_train, num_round)
    xxx = bst.predict(D_test)

    accuracy = 1.0*map(lambda x,y: 1 if x == y else 0, y_test, xxx).count(1)/len(y_test)
    accs.append(accuracy)
    
    for pr in range(len(xxx)):
        if xxx[pr] != y_test[pr]:
#            print('pred:', inv_lable_dict[xxx[pr]])
#            print('true:', inv_lable_dict[y_test[pr]])
#            print(doc_texts[j[pr]][1], '\n')
            incorrect_classes[inv_lable_dict[y_test[pr]]] += 1
            incorrect_files[doc_texts[j[pr]][1]] += 1
            common_mistakes[(inv_lable_dict[xxx[pr]], inv_lable_dict[y_test[pr]])] += 1
            #print(doc_texts[j][2])
    
    print ('---------------------------------\n', 'Done')
    print ('Fold', cc, 'accuracy :\t', accuracy, '\n')
    cc += 1
cv_acc = sum(accs)/len(accs)
print ('%s-Fold cross-validation accuracy over %s classes for the basic BST model is:' % (cv.n_splits, len(lable_dict)), cv_acc)

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.276000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Fold 0 ...


AttributeError: 'list' object has no attribute 'lower'

C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.283000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.288000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.292000. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
C:\Users\Artur_Zahreba\AppData\Local\Continuum\Anaconda3\lib\site-packages\jupyter_client\jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-06-28 16:31:00.295000. Please add timezone 

In [ ]:
_stopwords = set(stopwords.words('english') + list(punctuation) + 
                 ["pg.com", "from", "sent", "cc", "please", "thank", "subject"])

def getWords(text):
    word_sent = [word for word in word_tokenize(text) if word.lower() not in _stopwords]

    return word_sent

In [ ]:
approvers = ["Meng, FanRu", "Liang, Anna"]

def getApprovalText(headers):
    texts = []
    for h in headers:
        c = h.nextSibling
        
        while getattr(c, 'name', None) == 'p':
            text = c.text.encode('ascii', 'replace').decode('utf-8').replace("?","").strip()
            texts.append(text)
            c = c.nextSibling

#     return ' '.join(getWords(' '.join(texts).lower()))
    return ' '.join([t.strip() for t in texts]).strip()

def getDocText(filename):
    with open(docs_path + '/' + filename, 'rb') as fp:
        msg = BytesParser(policy=policy.default).parse(fp)
        msg_text = msg.get_body(preferencelist=('html', 'plain')).get_content()

        soup = BeautifulSoup(str(msg_text), "lxml")

    #     print(len(soup.findAll('div', text = approvers)))
        headers = set()
        
        wordsection = soup.find('div')
        
#         print(wordsection)
        
        if wordsection != None:
            for candidate in wordsection.findAll('div'):
                if candidate.div != None and any("From: "+ appr in candidate.text for appr in approvers):
                    headers.add(candidate)
        fp.close()
        
        return getApprovalText(headers)

In [ ]:
doc_texts

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(
    max_df = 0.5,
    min_df = 2,
    stop_words = 'english'
)

In [ ]:
X = vectorizer.fit_transform(doc_texts)
X

In [ ]:
from sklearn.cluster import KMeans

number_of_classes = 2

km = KMeans(
    n_clusters = number_of_classes, 
    init = 'k-means++', 
    max_iter = 500, 
    n_init = 1, 
    verbose = True
)
km

In [ ]:
km.fit(X)

In [ ]:
np.unique(km.labels_, return_counts=True)

In [ ]:
text = {}
for i, cluster in enumerate(km.labels_):
    oneDocument = doc_texts[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument
km.labels_

In [ ]:
keywords = {}
counts = {}

for cluster in range(number_of_classes):
    word_sent = word_tokenize(text[cluster])
#     word_sent = [word for word in word_sent if word.lower() not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(20, freq, key=freq.get)
    counts[cluster]=freq
keywords

In [ ]:
unique_keys = {}

for cluster in range(2):   
    other_clusters = list(set(range(2)) - set([cluster]))
    keys_other_clusters = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[0]]))
    unique = set(keywords[cluster]) - keys_other_clusters
    unique_keys[cluster] = nlargest(10, unique, key=counts[cluster].get)
    
unique_keys

In [ ]:
approval_text = "I approve on behalf of Anna"

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3)
classifier.fit(X, km.labels_)

In [ ]:
test = vectorizer.transform([approval_text.encode('ascii', errors='ignore').decode('utf8')])

In [ ]:
classifier.predict(test)

In [ ]:
# If we want to print a priview of the message content, we can extract whatever
# the least formatted payload is and print the first three lines.  Of course,
# if the message has no plain text part printing the first three lines of html
# is probably useless, but this is just a conceptual example.
simplest = msg.get_body(preferencelist=('related', 'html', 'plain'))
print()
email_full_text = ''.join(simplest.get_content().splitlines(keepends=True)[:120])
print(email__full_text)